In [13]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

base_url = 'https://www.musicomh.com/classical/reviews-classical/page/'
reviews=[]

In [ ]:
for page in range(104,105):
    print(page)
    url = base_url + str(page)
    r  = requests.get(url)
    soup = BeautifulSoup(r.text, "html5lib")
    listings = soup.find_all('h2', attrs={"class": "title"}) + soup.find_all('div', attrs={"class": "image_text_box_index"})        

    for listing in listings:
        
        review = []        
        listing_url = listing.find('a').get('href')        
        r     = requests.get(listing_url)
        soup  = BeautifulSoup(r.text, "html5lib")
        
        try:
            posted = soup.find(attrs={"class": "date time published"}).get_text()
        except:
            posted = '31 Dec 2099'
        date   = datetime.strptime(posted, '%d %b %Y').date()        
        
        try:
            info  = soup.find('div', attrs={"id": "post_title"}).find('h1').get_text()
            city  = info.split(',')[-1].strip()
            work  = info.split('@')[0].strip()
            if len(info.split('@')) > 1:
                venue = info.split('@')[1].split(',')[0].strip()
            else:
                venue = ''
        except:
            info = listing_url
            city = ''
            work = ''
            venue = ''

        try:
            rating = soup.find('div', attrs={"id": "post_title"}).parent.find('img')
            rating = rating.get('src').split('/')[-1].split('.')[0].replace('stars', '').replace('-', '.')
        except:
            rating = ''
            
        text  = soup.find('div', attrs={"class": "entry"}).find_all('p')
        full_text=''
        for w in text:
            if w.get('class')==None:
                full_text += w.get_text()
        
        review.append(info)
        review.append(work)
        review.append(venue)
        review.append(city)
        review.append(rating)        
        review.append(full_text)
        reviews.append(review)

import pandas as pd
headers = ['title', 'work', 'venue', 'city', 'rating', 'full_text']
df = pd.DataFrame(reviews, columns=headers)
df.to_csv('../../data/processed/reviews/musicomh.csv')